
# CC3039 — Modelación y Simulación  
## Laboratorio 7 — Parte Práctica 1 (SimPy, Clínica de 2 etapas)
- Joaquín Campos - 22155
- Sofía García - 22210
- Julio García Salas - 22076


In [ ]:
import simpy
import random
import statistics
import pandas as pd
import json

# ---- Semilla para reproducibilidad ----
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# ---- Parámetros globales ----
NUM_RECEPCIONISTAS = 1       
NUM_MEDICOS = 2              
TIEMPO_REGISTRO_PROMEDIO = 2.0   
TIEMPO_CONSULTA_PROMEDIO  = 7.0   

# TASA_LLEGADA_PACIENTES interpretada como "tasa por minuto" (1/5 significa ~un paciente cada 5 minutos)
TASA_LLEGADA_PACIENTES = 1.0 / 5.0

TIEMPO_SIMULACION = 120.0   

# ---- Colecciones globales de resultados ----
tiempos_de_espera_totales = []
esperas_registro = []
esperas_consulta = []

# Traza opcional por paciente 
traza_pacientes = []

IMPRIMIR_TRAZA = False
MAX_PACIENTES_EN_TRAZA = 10


In [ ]:
# 2) Proceso paciente y generador
def paciente(env, nombre, recepcionistas, medicos):
    llegada = env.now
    registro_espera = consulta_espera = 0.0
    registro_dur = consulta_dur = 0.0

    # Registro
    with recepcionistas.request() as req_reg:
        t_req = env.now
        yield req_reg
        registro_espera = env.now - t_req
        registro_dur = random.expovariate(1.0 / TIEMPO_REGISTRO_PROMEDIO)
        yield env.timeout(registro_dur)

    # Consulta
    with medicos.request() as req_med:
        t_req2 = env.now
        yield req_med
        consulta_espera = env.now - t_req2
        consulta_dur = random.expovariate(1.0 / TIEMPO_CONSULTA_PROMEDIO)
        yield env.timeout(consulta_dur)

    total_en_sistema = env.now - llegada
    tiempos_de_espera_totales.append(total_en_sistema)
    esperas_registro.append(registro_espera)
    esperas_consulta.append(consulta_espera)


def generador_pacientes(env, recepcionistas, medicos):
    i = 0
    while True:
        inter_arrival = random.expovariate(TASA_LLEGADA_PACIENTES)
        yield env.timeout(inter_arrival)
        i += 1
        env.process(paciente(env, f"Paciente {i}", recepcionistas, medicos))


In [ ]:
# 3) Función de ejecución y métricas
def ejecutar_simulacion(num_recepcionistas, num_medicos, tiempo_simulacion=TIEMPO_SIMULACION, semilla=RANDOM_SEED):
    random.seed(semilla)
    tiempos_de_espera_totales.clear()
    esperas_registro.clear()
    esperas_consulta.clear()

    env = simpy.Environment()
    recepcionistas = simpy.Resource(env, capacity=num_recepcionistas)
    medicos = simpy.Resource(env, capacity=num_medicos)

    env.process(generador_pacientes(env, recepcionistas, medicos))
    env.run(until=tiempo_simulacion)

    n = len(tiempos_de_espera_totales)
    return {
        "pacientes_atendidos": n,
        "prom_total_en_clinica": statistics.mean(tiempos_de_espera_totales) if n else 0.0,
        "max_total_en_clinica": max(tiempos_de_espera_totales) if n else 0.0,
        "prom_espera_registro": statistics.mean(esperas_registro) if esperas_registro else 0.0,
        "prom_espera_consulta": statistics.mean(esperas_consulta) if esperas_consulta else 0.0,
        "_tiempos": list(tiempos_de_espera_totales)  
    }


In [ ]:
# 4) Ejecutar escenarios y mostrar métricas
base = ejecutar_simulacion(1, 2)
medicos5 = ejecutar_simulacion(1, 5)
recep2 = ejecutar_simulacion(2, 2)

def to_row(nombre, d):
    return {
        "Escenario": nombre,
        "Pacientes": d["pacientes_atendidos"],
        "Prom Total (min)": round(d["prom_total_en_clinica"], 3),
        "Max Total (min)": round(d["max_total_en_clinica"], 3),
        "Prom Espera Registro (min)": round(d["prom_espera_registro"], 3),
        "Prom Espera Consulta (min)": round(d["prom_espera_consulta"], 3),
    }

import pandas as pd
df_comp = pd.DataFrame([
    to_row("Base (R=1, M=2)", base),
    to_row("Médicos=5 (R=1)", medicos5),
    to_row("Recep=2 (M=2)", recep2),
])
print("=== Tabla comparativa ===")
print(df_comp.to_string(index=False))

# Métricas clave adicionales 
delta_medicos = base['prom_total_en_clinica'] - medicos5['prom_total_en_clinica']
delta_recep   = base['prom_total_en_clinica'] - recep2['prom_total_en_clinica']

bloque = {
    "Q1": {
        "prom_total_en_clinica": round(base['prom_total_en_clinica'], 3),
        "prom_espera_registro": round(base['prom_espera_registro'], 3),
        "prom_espera_consulta": round(base['prom_espera_consulta'], 3)
    },
    "Q2": {
        "delta_prom_total_vs_base_medicos5": round(delta_medicos, 3)
    },
    "Q3": {
        "delta_prom_total_vs_base_recep2": round(delta_recep, 3)
    }
}

print("\n=== Bloque de resultados ===")
print(json.dumps(bloque, indent=2, ensure_ascii=False))



=== Tabla comparativa ===
      Escenario  Pacientes  Prom Total (min)  Max Total (min)  Prom Espera Registro (min)  Prom Espera Consulta (min)
Base (R=1, M=2)         23            10.816           29.282                       0.819                       2.253
Médicos=5 (R=1)         26             9.016           28.337                       1.285                       0.000
  Recep=2 (M=2)         22            11.107           29.282                       0.002                       2.714

=== Bloque de resultados ===
{
  "Q1": {
    "prom_total_en_clinica": 10.816,
    "prom_espera_registro": 0.819,
    "prom_espera_consulta": 2.253
  },
  "Q2": {
    "delta_prom_total_vs_base_medicos5": 1.8
  },
  "Q3": {
    "delta_prom_total_vs_base_recep2": -0.291
  }
}


**Base de datos usada para respuestas:** métricas provistas por el equipo (JSON).  
**Horizonte:** 120 min. **Llegadas:** Poisson. **Servicios:** exponenciales.

---

### Q1) Línea base (R=1, M=2)
- **Etapa con mayor espera promedio:** **Consulta**  
  (Registro = 0.82 min, Consulta = 2.25 min).
- **Tiempo promedio total en clínica:** **10.82 min**.

**Interpretación:** La espera se concentra en **consulta**, indicando mayor presión sobre el recurso **médico** bajo el régimen de carga actual.

---

### Q2) Aumentar médicos a 5 (R=1, M=5)
- **Cambio en el tiempo promedio total vs. base:** **↓ 1.80 min**  
  (promedio estimado ≈ 9.02 min).
- **Cuello de botella inferido:** **Médicos (consulta)**, consistente con Q1.

**Interpretación:** Incrementar la capacidad de **consulta** reduce sustancialmente el tiempo total en el sistema.

---

### Q3) Aumentar recepcionistas a 2 (R=2, M=2)
- **Cambio vs. base (promedio total):** **-0.29 min**  
  (promedio estimado ≈ 11.11 min).  
  *Nota:* un valor negativo implica **empeoramiento** ligero respecto a la base.
- **Intervención más efectiva:** **Incrementar médicos a 5** (mejor que aumentar recepcionistas a 2).

**Interpretación:** Con los parámetros actuales, **registro no es el limitante** principal; el cuello de botella permanece en **consulta**.

---

### Q4) Recomendación profesional (considerando menor costo de recepcionistas)
- Priorizar **refuerzo de médicos** (consulta), idealmente en **horas pico** con **turnos parciales/solapamientos**.
- Complementar con **gestión de citas** y/o **triage administrativo** para **alisar llegadas** y evitar saturaciones puntuales.
- Monitorear por franja y, si cambia el patrón de demanda, **re-evaluar** capacidad de registro.

> Nota metodológica: Por tratarse de un sistema estocástico, se recomienda reportar **promedios + P50/P90** a partir de **múltiples réplicas**.
